In [1]:
import os
# CUDAVISIBLE DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import transformers

print(torch.__version__)
print(torch.cuda.is_available())

2.9.1+cu128
True


In [2]:
from snapkv.monkeypatch.monkeypatch import replace_llama, replace_mistral, replace_mixtral

In [3]:
replace_mistral()

In [4]:
from fastchat.model import load_model, get_conversation_template

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    local_files_only=True, 
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
    use_flash_attention_2=True
    )

/home/ymz/anaconda3/envs/snapkv/lib/python3.10/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The model was loaded with use_flash_attention_2=True, which is deprecated and may be removed in a future release. Please use `attn_implementation="flash_attention_2"` instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    use_fast=False,          # 关键：不用 fast 版本
    local_files_only=True    # 如果你已经把模型 + tokenizer 一起下好了就用 True
                             # 不确定有没有 tokenizer 文件，就先去掉这一行，让它联网下
)

In [7]:
with open('snapkv.txt', 'r') as f:
    content = f.read().strip()

In [8]:
question = "\n What is the repository of SnapKV?"

In [9]:
conv = get_conversation_template("mistralai/Mistral-7B-Instruct-v0.2")
conv.messages = []
conv.append_message(conv.roles[0],content + question)
# conv.append_message(conv.roles[0],"Who is Kobe Bryant?")
conv.append_message(conv.roles[1], None)

In [10]:
prompt = conv.get_prompt()

In [11]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [12]:
input_ids_len = input_ids.size(1)
print(input_ids_len)

17395


In [13]:
outputs = model.generate(input_ids.cuda(), max_new_tokens=200, do_sample=False)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [14]:
print(tokenizer.decode(outputs[0][input_ids_len:], skip_special_tokens=True))

The repository of SnapKV is available at <https://github.com/FasterDecoding/SnapKV>.
